In [102]:
import pandas as pd

In [103]:
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = './testapp-60fcd-f547300981c5.json'
VIEW_ID = '170130817'

def initialize_analyticsreporting():
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)
  analytics = build('analyticsreporting', 'v4', credentials=credentials)
  return analytics

def get_report(analytics, pageTokenVar):
  return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '700daysAgo', 'endDate': 'yesterday'}],
          'metrics': [{'expression': 'ga:pageviews'},{'expression': 'ga:entrances'}],
          'dimensions': [{'name': 'ga:pagePath'}],
          'dimensionFilterClauses': [{
                    'operator': "AND",
                    'filters': [
                          {
                          'dimensionName': 'ga:pagePath',
                          'operator': 'REGEXP',
                          'expressions': [
                            '.*id[0-9]{7}.htm'
                          ]},
                          {
                          'dimensionName': 'ga:pagePath',
                          'operator': 'REGEXP',
                          'expressions': [
                            '^[^?&]*$'
                          ]}
                    ]    
          }],
          'metricFilterClauses': [{ 
              "filters": [
                        {
                          'metricName': 'ga:pageviews',
                          'not': 'false',
                          'operator': 'LESS_THAN',
                          'comparisonValue': '6'
                        }
                    ]   
          }],
          'pageSize': 100000,
          'pageToken': pageTokenVar,
          'samplingLevel': 'LARGE'
          
        }],
        'useResourceQuotas': 'true'
      }
  ).execute()
    
def handle_report(analytics,pagetoken,rows):  

    response = get_report(analytics, pagetoken)

    #Header, Dimentions Headers, Metric Headers 
    columnHeader = response.get("reports")[0].get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

    #Pagination
    pagetoken = response.get("reports")[0].get('nextPageToken', None)
    
    #Rows
    rowsNew = response.get("reports")[0].get('data', {}).get('rows', [])
    rows = rows + rowsNew
    print("len(rows): " + str(len(rows)))

    #Recursivly query next page
    if pagetoken != None:
        return handle_report(analytics,pagetoken,rows)
    else:
        return rows

def main():    
    analytics = initialize_analyticsreporting()
    
    global dfanalytics
    dfanalytics = []

    rows = []
    rows = handle_report(analytics,'0',rows)

    dfanalytics = pd.DataFrame(list(rows))

if __name__ == '__main__':
  main()

len(rows): 100000
len(rows): 200000
len(rows): 223174


In [104]:
tags = dfanalytics['dimensions'].apply(pd.Series)
tags = tags.rename(columns = lambda x : 'pagePath')
dfanalytics = pd.concat([dfanalytics[:], tags[:]], axis=1)  

In [105]:
#Drop columns about performance. Don't care if there were 3 or 4 pageviews 
dfanalytics = dfanalytics.drop("dimensions",1)
dfanalytics = dfanalytics.drop("metrics",1)

In [106]:
#Generate a short URL for the crawl with just the ID
dfanalytics['ID_GA'] = dfanalytics['pagePath'].str.extract('(?:.*id)([0-9]+)(?:.html)', expand=False)
dfanalytics['ShortURL'] = "https://www.blick.ch/"+dfanalytics['ID_GA']+'/'

In [107]:
#Drop duplicates, after droping pagePath 
dfanalytics = dfanalytics.drop("pagePath",1)
dfanalytics = dfanalytics.drop_duplicates()
dfanalytics

,ID_GA,ShortURL
0,5378225,https://www.blick.ch/5378225/
1,7091834,https://www.blick.ch/7091834/
2,7252099,https://www.blick.ch/7252099/
3,7277672,https://www.blick.ch/7277672/
4,7393854,https://www.blick.ch/7393854/
5,7987322,https://www.blick.ch/7987322/
6,8943756,https://www.blick.ch/8943756/
7,8943976,https://www.blick.ch/8943976/
8,2153815,https://www.blick.ch/2153815/
9,8441776,https://www.blick.ch/8441776/


In [108]:
#Export crawl
#dfanalyticsExport = dfanalytics.drop("ID_GA",1)
dfanalyticsExport.to_csv('dfanalyticsExport.txt', header=None, index=None, sep=' ', mode='a')

In [109]:
#Crawl here
#Import crawl 
dfCrawled = pd.DataFrame(pd.read_excel('./custom_search_all.xlsx', header=0, sheet_name='1 - All'))
dfCrawled

,Address,Content,Status Code,Status,"Contains: '""article-text-box""'","Contains: '""video-player""'",Contains: '<picture',"Contains: '""article-body'"
0,https://www.blick.ch/news/schweiz/parteien-sp-...,text/html; charset=utf-8,200,OK,0,0,8,8
1,https://www.blick.ch/people-tv/musik/menschenr...,text/html; charset=utf-8,200,OK,0,0,9,2
2,https://www.blick.ch/people-tv/international/a...,text/html; charset=utf-8,200,OK,0,0,13,5
3,https://www.blick.ch/news/schweiz/schneebrett-...,text/html; charset=utf-8,200,OK,0,0,8,2
4,https://www.blick.ch/people-tv/schweiz/mode-bl...,text/html; charset=utf-8,200,OK,0,1,10,6
5,https://www.blick.ch/sport/eishockey/nla/ein-w...,text/html; charset=utf-8,200,OK,0,0,10,3
6,https://www.blick.ch/news/ausland/ex-polizist-...,text/html; charset=utf-8,200,OK,0,0,9,1
7,https://www.blick.ch/news/schweiz/bern/sie-kli...,text/html; charset=utf-8,200,OK,0,0,8,6
8,https://www.blick.ch/news/ausland/kurios/hirnv...,text/html; charset=utf-8,200,OK,0,0,11,9
9,https://www.blick.ch/news/wirtschaft/belgier-b...,text/html; charset=utf-8,200,OK,0,0,8,7


In [110]:
#Rename columns
dfCrawled = dfCrawled.rename(columns={"Contains: \'\"article-text-box\"\'": "article-text-box", 
                                         "Contains: \'\"video-player\"\'":"video-player", 
                                         "Contains: \'<picture\'":"picture", 
                                         "Contains: \'\"article-body\'":"article-body" })

In [111]:
#Getting the Id for later merge
dfCrawled['ID_Crawl'] = dfCrawled['Address'].str.extract('(?:.*id)([0-9]+)(?:.html)', expand=False)

In [112]:
dfCrawled = dfCrawled.drop("Status Code",1)
dfCrawled = dfCrawled.drop("Status",1)
dfCrawled = dfCrawled.drop("Content",1)

In [113]:
#Merge is important because of possible redirects to useful articels 
dfMerged = pd.merge(dfanalytics, dfCrawled, left_on='ID_GA', right_on='ID_Crawl', how='outer')
dfMerged = dfMerged[dfMerged['ID_Crawl'].notnull()]
dfMerged = dfMerged[dfMerged['ID_GA'].notnull()]
dfMerged = dfMerged.drop("ShortURL",1)
dfMerged = dfMerged.drop("ID_Crawl",1)
dfMerged


,ID_GA,Address,article-text-box,video-player,picture,article-body
0,5378225,https://www.blick.ch/people-tv/seth-rogen-wirb...,0.0,1.0,8.0,0.0
1,7091834,https://www.blick.ch/news/ausland/new-yorker-r...,0.0,1.0,8.0,8.0
2,7252099,https://www.blick.ch/news/ausland/die-behoerde...,0.0,1.0,8.0,0.0
3,7277672,https://www.blick.ch/news/ausland/enzym-zerset...,0.0,1.0,8.0,0.0
4,7393854,https://www.blick.ch/news/ausland/kenia-hat-da...,0.0,1.0,8.0,0.0
9,8441776,https://www.blick.ch/people-tv/tv/bachelorette...,0.0,0.0,13.0,8.0
10,9005034,https://www.blick.ch/sport/fussball/internatio...,0.0,1.0,8.0,0.0
11,8946692,https://www.blick.ch/sport/fussball/internatio...,0.0,1.0,8.0,0.0
12,7068779,https://www.blick.ch/sport/videoformate/best-o...,0.0,1.0,8.0,0.0
13,8958308,https://www.blick.ch/sport/fussball/1-1-sieg-g...,0.0,1.0,8.0,0.0


In [114]:
#dfMerged.to_csv('mergedNew.csv', sep='\t')

In [115]:
#Usecases Pure Bullshit: No video, No images, No article text, no textbox
dfPureBullshit = dfMerged
dfPureBullshit = dfPureBullshit[dfPureBullshit['article-text-box'] == 0]
dfPureBullshit = dfPureBullshit[dfPureBullshit['article-body'] == 0]
dfPureBullshit = dfPureBullshit[dfPureBullshit['picture'] < 9]
dfPureBullshit = dfPureBullshit[dfPureBullshit['video-player'] == 0]
dfPureBullshit

,ID_GA,Address,article-text-box,video-player,picture,article-body
165,1443922,https://www.blick.ch/aktuell/15-12-swiss-flieg...,0.0,0.0,8.0,0.0
904,7410126,https://www.blick.ch/auto/abgefahren/facebook-...,0.0,0.0,8.0,0.0
1073,1796059,https://www.blick.ch/auto/autosalon/direkt-vom...,0.0,0.0,8.0,0.0
1105,6305932,https://www.blick.ch/auto/autosalon/blick-auto...,0.0,0.0,8.0,0.0
1123,2714035,https://www.blick.ch/auto/autosalon/interaktiv...,0.0,0.0,8.0,0.0
1127,2717233,https://www.blick.ch/auto/autosalon/interaktiv...,0.0,0.0,8.0,0.0
1134,2716437,https://www.blick.ch/auto/autosalon/interaktiv...,0.0,0.0,8.0,0.0
1169,4770422,https://www.blick.ch/auto/autosalon/multimedia...,0.0,0.0,8.0,0.0
1259,8070433,https://www.blick.ch/auto/autosalon/news/blick...,0.0,0.0,8.0,0.0
1282,2227668,https://www.blick.ch/auto/autosalon/news/mehr-...,0.0,0.0,8.0,0.0


In [116]:
dfNoTextButSomeImages = dfMerged
dfNoTextButSomeImages = dfNoTextButSomeImages[dfNoTextButSomeImages['article-text-box'] == 0]
dfNoTextButSomeImages = dfNoTextButSomeImages[dfNoTextButSomeImages['article-body'] == 0]
dfNoTextButSomeImages = dfNoTextButSomeImages[dfNoTextButSomeImages['picture'] > 10]
dfNoTextButSomeImages

,ID_GA,Address,article-text-box,video-player,picture,article-body
83,8961535,https://www.blick.ch/people-tv/fotostory/32-ja...,0.0,0.0,13.0,0.0
117,8755696,https://www.blick.ch/people-tv/fotostory/ab-fr...,0.0,0.0,12.0,0.0
1179,4751915,https://www.blick.ch/auto/autosalon/news/augen...,0.0,0.0,13.0,0.0
1199,2226057,https://www.blick.ch/auto/autosalon/news/die-m...,0.0,0.0,13.0,0.0
1325,1797530,https://www.blick.ch/auto/autosalon/promis/die...,0.0,0.0,13.0,0.0
2095,1721007,https://www.blick.ch/auto/news_n_trends/detroi...,0.0,0.0,13.0,0.0
2104,2049991,https://www.blick.ch/auto/news_n_trends/die-sc...,0.0,0.0,13.0,0.0
2183,4379858,https://www.blick.ch/auto/news_n_trends/fantas...,0.0,0.0,13.0,0.0
2259,1677955,https://www.blick.ch/auto/news_n_trends/gesche...,0.0,0.0,13.0,0.0
2288,1867903,https://www.blick.ch/auto/news_n_trends/miss-t...,0.0,0.0,13.0,0.0


In [117]:
dfNoText = dfMerged
dfNoText = dfNoText[dfNoText['article-text-box'] == 0]
dfNoText = dfNoText[dfNoText['article-body'] == 0]
dfNoText

,ID_GA,Address,article-text-box,video-player,picture,article-body
0,5378225,https://www.blick.ch/people-tv/seth-rogen-wirb...,0.0,1.0,8.0,0.0
2,7252099,https://www.blick.ch/news/ausland/die-behoerde...,0.0,1.0,8.0,0.0
3,7277672,https://www.blick.ch/news/ausland/enzym-zerset...,0.0,1.0,8.0,0.0
4,7393854,https://www.blick.ch/news/ausland/kenia-hat-da...,0.0,1.0,8.0,0.0
10,9005034,https://www.blick.ch/sport/fussball/internatio...,0.0,1.0,8.0,0.0
11,8946692,https://www.blick.ch/sport/fussball/internatio...,0.0,1.0,8.0,0.0
12,7068779,https://www.blick.ch/sport/videoformate/best-o...,0.0,1.0,8.0,0.0
13,8958308,https://www.blick.ch/sport/fussball/1-1-sieg-g...,0.0,1.0,8.0,0.0
14,8945118,https://www.blick.ch/sport/eishockey/nhl/1-sch...,0.0,1.0,8.0,0.0
16,7074460,https://www.blick.ch/news/ausland/10-verletzte...,0.0,1.0,8.0,0.0


In [119]:
dfNoTextButTextbox = dfMerged
dfNoTextButTextbox = dfNoTextButTextbox[dfNoTextButTextbox['article-text-box'] != 0]
dfNoTextButTextbox = dfNoTextButTextbox[dfNoTextButTextbox['article-body'] == 0]
dfNoTextButTextbox

,ID_GA,Address,article-text-box,video-player,picture,article-body
227,1660333,https://www.blick.ch/aktuell/das-war-der-chat-...,1.0,0.0,9.0,0.0
352,1660346,https://www.blick.ch/aktuell/das-chat-protokol...,1.0,0.0,8.0,0.0
3648,2926517,https://www.blick.ch/auto/tests/chevrolet-corv...,2.0,1.0,10.0,0.0
5369,2143894,https://www.blick.ch/digital/die-welt-der-tech...,7.0,0.0,16.0,0.0
5463,3386849,https://www.blick.ch/digital/gadget-news-von-d...,4.0,0.0,16.0,0.0
5634,2643496,https://www.blick.ch/digital/games/game-gegen-...,2.0,0.0,15.0,0.0
5635,3710102,https://www.blick.ch/digital/games/game-gegen-...,3.0,0.0,16.0,0.0
5636,3368173,https://www.blick.ch/digital/games/game-gegen-...,3.0,0.0,16.0,0.0
5638,2691578,https://www.blick.ch/digital/games/game-gegen-...,3.0,0.0,16.0,0.0
5639,2615893,https://www.blick.ch/digital/games/game-gegen-...,2.0,0.0,15.0,0.0
